In [ ]:
#load necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from mlxtend.evaluate import bootstrap_point632_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, matthews_corrcoef, roc_auc_score, plot_roc_curve

In [ ]:
#Read file
#samples should be in rows and features in columns
#data already normalized from metaboanalyst 
df = pd.read_csv("path_to_file")

#prepare and encode data
data = df.values
X = data[:, 1:-1]
y = data[:, -1].astype(str)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

#check for class imbalance
print('\nratio of Healthy and Infected = ', sum(y)/len(y))

In [ ]:
#define model
RFmodel = RandomForestClassifier(n_estimators=100, criterion='gini', min_samples_split=2,
                                 max_features='sqrt', max_leaf_nodes=None)

In [ ]:
#optimize hyperparameters

max_depth = list(range(1,20))
min_samples_leaf = np.arange(0.01,1,0.01)
param_dict = {"max_depth":max_depth, 'min_samples_leaf': min_samples_leaf}
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)
grid_search = GridSearchCV(RFmodel, param_grid=param_dict, cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X, y)
print('best parameter', grid_search.best_params_, 'accuracy', grid_search.best_score_)

#capture best hyperparameter
max_depth_best = grid_search.best_params_['max_depth']
min_samples_leaf_best = grid_search.best_params_['min_samples_leaf']

In [ ]:
#redefine model with best hyperparameter
RFmodel_best = RandomForestClassifier(n_estimators=100, criterion='gini', min_samples_split=2, max_depth=max_depth_best,
                                      min_samples_leaf = min_samples_leaf_best, max_features='sqrt', max_leaf_nodes=None)

#fit the model
RFmodel_best.fit(X, y)

#plot ROC curve
plot_roc_curve(RFmodel_best, X, y)

In [ ]:
# 632+bootstrap scoring metrics

metrics = {'Accuracy': accuracy_score, 'Precision': precision_score, 'Recall': recall_score,
           'F1': f1_score, 'matthews correlation coffecient': matthews_corrcoef, 'ROC AUC': roc_auc_score}

for name, metric in metrics.items():
    scores = bootstrap_point632_score(RFmodel_best, X, y, n_splits=200, method='.632+', scoring_func=metric,
                                  predict_proba=False, random_seed=None, clone_estimator=True)
    mean_score = np.mean(scores)
    print(name, '%.2f' % mean_score)
    # Confidence interval
    lower = np.percentile(scores, 2.5)
    upper = np.percentile(scores, 97.5)
    print('95%% Confidence interval: [%.2f, %.2f]' % (100*lower, 100*upper))
    print('\n')

In [ ]:
#plot important features
important_features = RFmodel_best.feature_importances_
plt.bar([x for x in range(len(important_features))], important_features)
plt.show()

In [ ]:
#save important features to a .csv
df_important_features = pd.DataFrame(enumerate(important_features))
df_important_features_sorted = df_important_features.sort_values(1, axis=0, ascending=False)
df_important_features_sorted_indexes = df_important_features_sorted[0]
feature_labels = df.columns[1:-1]
df_important_features_sorted_labels = feature_labels[df_important_features_sorted_indexes]
df_important_features_sorted['label'] = df_important_features_sorted_labels
df_important_features_sorted = df_important_features_sorted.rename(columns={0:'index', 1:'importance'})
df_important_features_sorted.to_csv("important features from RF with max_depth 15 and min_samples_leaf 0.02.csv")
df_important_features_sorted